# Подготовка базы данных

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('window.db')

In [3]:
cursor = con.cursor()

In [4]:
sql = '''

create table employees (
    id integer primary key,
    name varchar(50),
    city varchar(50),
    department varchar(50),
    salary integer
);

insert into employees
(id, name, city, department, salary)
values
(24, 'Марина', 'Москва', 'it', 104),
(21, 'Елена', 'Самара', 'it', 84),
(22, 'Ксения', 'Москва', 'it', 90),
(25, 'Иван', 'Москва', 'it', 120),
(23, 'Леонид', 'Самара', 'it', 104),
(11, 'Дарья', 'Самара', 'hr', 70),
(12, 'Борис', 'Самара', 'hr', 78),
(31, 'Вероника', 'Москва', 'sales', 96),
(33, 'Анна', 'Москва', 'sales', 100),
(32, 'Григорий', 'Самара', 'sales', 96);

create table expenses (
    year integer,
    month integer,
    income integer,
    expense integer
);

insert into expenses
(year, month, income, expense)
values
(2020, 1, 94, 82),
(2020, 2, 94, 75),
(2020, 3, 94, 104),
(2020, 4, 100, 94),
(2020, 5, 100, 99),
(2020, 6, 100, 105),
(2020, 7, 100, 95),
(2020, 8, 100, 110),
(2020, 9, 104, 104);'''

In [5]:
cursor.executescript(sql)

OperationalError: table employees already exists

In [6]:
sql = '''select * from employees'''

pd.read_sql(sql, con)

,id,name,city,department,salary
0,11,Дарья,Самара,hr,70
1,12,Борис,Самара,hr,78
2,21,Елена,Самара,it,84
3,22,Ксения,Москва,it,90
4,23,Леонид,Самара,it,104
5,24,Марина,Москва,it,104
6,25,Иван,Москва,it,120
7,31,Вероника,Москва,sales,96
8,32,Григорий,Самара,sales,96
9,33,Анна,Москва,sales,100


In [7]:
# Напишем функцию, чтобы удобнее было возвращать
def select(sql):
    return pd.read_sql(sql, con)

In [8]:
select(sql)

,id,name,city,department,salary
0,11,Дарья,Самара,hr,70
1,12,Борис,Самара,hr,78
2,21,Елена,Самара,it,84
3,22,Ксения,Москва,it,90
4,23,Леонид,Самара,it,104
5,24,Марина,Москва,it,104
6,25,Иван,Москва,it,120
7,31,Вероника,Москва,sales,96
8,32,Григорий,Самара,sales,96
9,33,Анна,Москва,sales,100


# Задача 1

**Есть таблица сотрудников employees. Мы хотим для каждого сотрудника увидеть, сколько процентов составляет его зарплата от общего фонда труда по городу:**

In [9]:
sql = '''
select t.name, t.city,
t.salary,
sum(t.salary) over w as fund,
round(t.salary*100.0 / sum(t.salary) over w) as perc
from employees as t

window w as (
    partition by t.city
)

order by t.city, t.salary
'''
select(sql)

,name,city,salary,fund,perc
0,Ксения,Москва,90,510,18.0
1,Вероника,Москва,96,510,19.0
2,Анна,Москва,100,510,20.0
3,Марина,Москва,104,510,20.0
4,Иван,Москва,120,510,24.0
5,Дарья,Самара,70,432,16.0
6,Борис,Самара,78,432,18.0
7,Елена,Самара,84,432,19.0
8,Григорий,Самара,96,432,22.0
9,Леонид,Самара,104,432,24.0


# Задача 2

**Есть таблица сотрудников employees. Мы хотим для каждого сотрудника увидеть:**

* сколько человек трудится в его отделе (emp_cnt);
* какая средняя зарплата по отделу (sal_avg);
* на сколько процентов отклоняется его зарплата от средней по отделу (diff).

In [10]:
sql = '''
select t.name, t.department, t.salary,
count(*) over w as emp_cnt,
avg(t.salary) over w as sal_avg,
round((t.salary - (avg(t.salary) over w))*100.0 / (avg(t.salary) over w)) as diff



from employees as t
window w as (

partition by t.department

)

order by t.department, t.salary, t.id

'''
select(sql)

,name,department,salary,emp_cnt,sal_avg,diff
0,Дарья,hr,70,2,74.000000,-5.0
1,Борис,hr,78,2,74.000000,5.0
2,Елена,it,84,5,100.400000,-16.0
3,Ксения,it,90,5,100.400000,-10.0
4,Леонид,it,104,5,100.400000,4.0
5,Марина,it,104,5,100.400000,4.0
6,Иван,it,120,5,100.400000,20.0
7,Вероника,sales,96,3,97.333333,-1.0
8,Григорий,sales,96,3,97.333333,-1.0
9,Анна,sales,100,3,97.333333,3.0


# Задача 3

**Расчет суммарной зарплаты по департаменту в городе. Обозначим как x. А суммарную зарплату по таблице обозначим как y**

In [12]:
sql = '''
select
  city,
  department,
  sum(salary) as dep_salary,
  sum(sum(salary)) over (partition by city) as x,
  sum(sum(salary)) over () as y
from employees
group by city, department
order by city, department;

'''
select(sql)

,city,department,dep_salary,x,y
0,Москва,it,314,510,942
1,Москва,sales,196,510,942
2,Самара,hr,148,432,942
3,Самара,it,188,432,942
4,Самара,sales,96,432,942
